This notebook contains evaluation of base T5 small Model trained on pubmed research paper dataset. ROUGE is used for measuring accuracy. The test example used is from BigPatent dataset.
https://huggingface.co/docs/transformers/model_doc/longt5#transformers.LongT5ForConditionalGeneration

This notebook has not been executed so scores are not mentioned.

In [ ]:
!pip install datasets
!pip install transformers
!pip install sentencepiece
!pip install rouge-score
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 4.5 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

#textiles/paper
ds = load_dataset("big_patent", "d")  # only 'a' CPC codes


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/10164 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/565 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/565 [00:00<?, ? examples/s]

Dataset big_patent downloaded and prepared to /root/.cache/huggingface/datasets/big_patent/d/2.1.2/bc8ec8bdf469c0da5fef04becd32bb3b0b34df0b0baa088ae1237628dd7a9caa. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
#import pandas as pd
# put data into dataFrames and csv
#df_train = pd.DataFrame(ds['train'])
#df_val = pd.DataFrame(ds['validation'])
#df_test = pd.DataFrame(ds['test'])
#df_train.to_csv("train.csv",index=False)
#df_val.to_csv("val.csv",index=False)
#df_test.to_csv("test.csv",index=False)

In [ ]:
# cleaning text
import re

def remove_sentence_with_fig(paragraph):


    # first we remove . after FIG
    paragraph = re.sub(r"FIG.","FIG",paragraph)

    # Split the paragraph into sentences
    sentences = re.split(r'(?<=[.!?])\s+', paragraph)

    # Remove sentences containing "FIG."
    filtered_sentences = [sentence for sentence in sentences if "FIG" not in sentence]

    # Join the remaining sentences to form the updated paragraph
    updated_paragraph = ' '.join(filtered_sentences)

    return updated_paragraph

def clean_text(text):

    # replace ° degree with
    text = re.sub(r"°", " degree", text)

    # replace US Patent
    text = re.sub(r" U.S. Pat. No. [0-9,]+", "Patent", text)

    # replace semi-colon with full-stop
    text = re.sub(r";", ".", text)
    # Remove special characters and symbols
    text = re.sub(r"[^a-zA-Z0-9\s.:,-]", "", text)

    # Normalize whitespace
    text = re.sub(r"\s+", " ", text)


    text = text.strip()

    return text


text = ds['test'][0]["description"]
cleaned_text = clean_text(text)
text = remove_sentence_with_fig(cleaned_text)

print(cleaned_text)
print(text)

CROSS-REFERENCE TO RELATED APPLICATIONS The present application claims priority from Australian Provisional Patent Application No 2006903417 filed on 26 Jun. 2006, the content of which is incorporated herein by reference. INTRODUCTION TO THE INVENTION This invention relates to the packaging of consumer products and relates particularly to products that are formed of pulp material, particularly pulp material formed of waste paper, waste timber, waste fabric material, virgin timber and other similar pulp material. The invention more particularly relates to high quality products carrying high definition printing or other decoration. BACKGROUND TO THE INVENTION The creation of compelling and high quality packaging for consumer durables is well established and is executed in a variety of forms and formats known in the prior art but with each of the prior art, the formats and methodologies having their own particular limitations. The consumer market demands increasing colour, vibrancy and no

In [ ]:
# divide the text into paragraphs of 500
text1 = text.split()
text_lst = []
for i in range(0,len(text1),500):
  tmp = " ".join(text1[i:i+500])
  text_lst.append(tmp)


In [ ]:
# exploring the dataset
print(ds['test'][0]["description"])

print("\n\n\n-------------")
print(ds['test'][0]["abstract"])

In [ ]:
# For cleaning predicted text
def clean_predicted_text(text):


    # replace semi-colon with full-stop
    text = re.sub(r";", ".", text)

    #replace new line character
    text = re.sub(r"<n>", "\n", text)

    #replace start of paragraph
    text = re.sub(r"<s>", "", text)

    # Remove special characters and symbols
    text = re.sub(r"[^a-zA-Z0-9\s.:,-]", "", text)

    # Normalize whitespace
    text = re.sub(r"\s+", " ", text)




    return text

In [ ]:
import torch
from transformers import pipeline


# summary expected length
#expected_length = len(ds['test'][0]["abstract"])//2

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("t5-small")

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

# you can change `block_size` & `num_random_blocks` like this:

predictions = []
#pred_summary_length = expected_length//len(text_lst)
summarizer = pipeline("summarization",model=model,tokenizer=tokenizer)
for i in range(0,len(text_lst)):
  tmp = text_lst[i]
  prediction = summarizer(tmp)[0]["summary_text"]
  predictions.append(prediction)




Token indices sequence length is longer than the specified maximum sequence length for this model (613 > 512). Running this sequence through the model will result in indexing errors
Your max_length is set to 200, but your input_length is only 186. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=93)


In [ ]:
from rouge_score import rouge_scorer
import evaluate

#rouge = evaluate.load("rouge")
abstract = ds['test'][0]["abstract"]
predicted_abstract = " ".join(predictions)

#result = rouge.compute(predictions=predicted_abstract, references=abstract, use_stemmer=True)
scorer = rouge_scorer.RougeScorer(['rouge1','rouge2' ,'rougeL'], use_stemmer=True)
scores = scorer.score(abstract, predicted_abstract)

In [ ]:
predicted_abstract

'the present application claims priority from Australian Provisional Patent Application No 2006903417 filed on 26 Jun. 2006 . the invention relates particularly to products formed of pulp material, particularly pulp material formed of waste paper, waste timber, waste fabric material and virgin timber . the more complicated the printing and laminating involved in any manufacture of a packaging product, the more costly the end product results . a number of the perceived high quality treatments in cardboard and paper packaging require the use of processes that are not environmentally sustainable . the main process used for packaging materials is the Fourdrinier process . organically-derived plastics can also have problems, in particular the so called bio-polymers, which may not be as sustainable as they first appear . a number of high end products including perfumes, skin care products etc have been used in packaging because they can be brightly coloured and formed into eye catching shape

In [ ]:
#rouge scores
scores

{'rouge1': Score(precision=0.12737127371273713, recall=0.8867924528301887, fmeasure=0.22274881516587677),
 'rouge2': Score(precision=0.0407055630936228, recall=0.2857142857142857, fmeasure=0.07125890736342043),
 'rougeL': Score(precision=0.07181571815718157, recall=0.5, fmeasure=0.12559241706161137)}